In [6]:
#!/usr/bin/env python3

"""
Plots Karyotype data
"""
import sys
sys.path.insert(1, 'workflow/scripts/')
sys.path.insert(2, '../../workflow/scripts/')

import rnaseqpoptools as rnaseqpop
import plotly.express as px
import pandas as pd 

import allel
import numpy as np

In [7]:
# Read in parameters from snakemake
ploidy = 10
config_path = "../../config/config.yaml"
dataset = "Ag_Busia"
metadata_path = "../../config/samples_oldnames.tsv"
tag_snp_path = "../../resources/karyotype_tag_snps.csv"
wkdir = "../.."
contig_r = "2R"
contig_l = "2L"
inversions = ['2La']


In [8]:
df_tag_snps = pd.read_csv(tag_snp_path, sep=",")
df_tag_snps.head()

,inversion,contig,position,alt_allele
0,2La,2L,20528221,T
1,2La,2L,20542311,A
2,2La,2L,20542372,T
3,2La,2L,20542904,T
4,2La,2L,20546192,A


In [9]:
import yaml
with open(config_path) as params_file:
    config_params = yaml.safe_load(params_file)

selected_invs = config_params['VariantAnalysis']['karyotype']['inversions']
selected_invs = ['2La', '2Rb']

metadata = rnaseqpop.load_metadata(metadata_path)
metadata = metadata.sort_values(by='species')

In [10]:
def _karyotype_tags_n_alt(gt, alts, inversion_alts):
    n_sites = gt.shape[0]
    n_samples = gt.shape[1]

    # create empty array
    inv_n_alt = np.empty((n_sites, n_samples), dtype=np.int8)

    # for every site
    for i in range(n_sites):
        # find the index of the correct tag snp allele
        if all(alts[i] != inversion_alts[i]):
            inv_n_alt[i, :] = 0
        else:
            tagsnp_index = np.where(alts[i] == inversion_alts[i])[0] + 1

        for j in range(n_samples):
            # count alleles which == tag snp allele and store
            n_tag_alleles = np.sum(gt[i, j] == tagsnp_index[0])
            inv_n_alt[i, j] = n_tag_alleles

    return inv_n_alt

In [11]:
inv_dict = {contig_l:["2La"], contig_r:['2Rb', '2Rc_gam', '2Rc_col', '2Rd', '2Rj']}

dfs = []
for contig, invs in inv_dict.items():

    if not np.any([i in invs for i in selected_invs]):
        continue         

    vcf, geno, ac_subpops, pos, alts, depth, snpeff, subpops, samples = rnaseqpop.readAndFilterVcf(f"../../results/variantAnalysis/vcfs/{dataset}.{contig}.vcf.gz",
                                contig, 
                                samples=metadata, 
                                ploidy=ploidy,
                                )
    

    for inversion in invs:

        if inversion in selected_invs:

            df_tag_snps_inv = df_tag_snps.query("inversion == @inversion")
            inversion_pos = df_tag_snps_inv.position
            inversion_alts = df_tag_snps_inv.alt_allele.to_numpy()

            # subset to position of inversion tags
            mask, mask2 = pos.locate_intersection(inversion_pos)
            tag_alts = inversion_alts[mask2]
            alts_inv_rna = alts[mask]
            gn = geno.compress(mask, axis=0)

            # infer karyotype
            gn_alt = _karyotype_tags_n_alt(
                gt=gn, alts=alts_inv_rna, inversion_alts=tag_alts
            )
            is_called = gn.is_called()

            # calculate mean genotype for each sample whilst masking missing calls
            av_gts = np.mean(np.ma.MaskedArray(gn_alt, mask=~is_called), axis=0)
            total_sites = np.sum(is_called, axis=0)

            df = pd.DataFrame(
                {
                    "sample_id": samples,
                    "inversion": inversion,
                    f"karyotype_{inversion}_mean": av_gts.round(2),
                    f"karyotype_{inversion}_freq": (av_gts / ploidy).round(2),
                    f"karyotype_{inversion}_n_tag_snps": total_sites,
                },
                    )
            dfs.append(df.set_index('sample_id'))


-------------- Reading VCF for chromosome 2L --------------
------- Filtering VCF at QUAL=30 and missingness proportion of 0.6 -------
QUAL filter will retain 322603 SNPs retained out of 396426 for chromosome 2L
Missingness filter will retain 295851 SNPs out of 396426 for chromosome 2L
The combined filter will retain 229853 SNPs out of 396426 for chromosome 2L

-------------- Reading VCF for chromosome 2R --------------
------- Filtering VCF at QUAL=30 and missingness proportion of 0.6 -------
QUAL filter will retain 437795 SNPs retained out of 533960 for chromosome 2R
Missingness filter will retain 404904 SNPs out of 533960 for chromosome 2R
The combined filter will retain 318011 SNPs out of 533960 for chromosome 2R


In [12]:
df_karyo = pd.concat(dfs, axis=1).filter(like="karyo")
df_karyo = df_karyo.sort_values('sample_id')
df_karyo.to_csv(f"{wkdir}/results/karyotype/karyotypes.tsv", sep="\t")
df_karyo

,karyotype_2La_mean,karyotype_2La_freq,karyotype_2La_n_tag_snps,karyotype_2Rb_mean,karyotype_2Rb_freq,karyotype_2Rb_n_tag_snps
sample_id,,,,,,
BusiaParental1,3.41,0.34,27,0.66,0.07,44
BusiaParental2,2.52,0.25,27,0.41,0.04,44
BusiaParental3,3.58,0.36,26,0.51,0.05,43
BusiaParental4,2.42,0.24,26,1.07,0.11,42
BusiaParental5,4.52,0.45,27,0.70,0.07,43
BusiaParental6,4.30,0.43,27,0.34,0.03,44
BusiaSelected1,10.00,1.00,26,0.49,0.05,43
BusiaSelected2,10.00,1.00,27,0.36,0.04,44
BusiaSelected3,7.31,0.73,26,0.74,0.07,43


In [ ]:
df.

In [40]:
df.shape[1] * 300

600

### Karyotyping

**Output Directory:** <span style="color:gray;font-weight:bold">*results/karyotype/*</span>

**Rules**  

<span style="color:gray;font-weight:bold">
    
* *variantAnalysis.smk*
    * Karyotype  
    
</span> 


**Introduction** 

Chromosomal inversions are a type of structural variation in which a segment of a chromosome is inverted relative to the normal ancestral arrangement. In *Anopheles gambiae*, chromosomal inversions have been extensively studied due to their role in the evolution and adaptation of the species. These inversions limit recombination in heterokaryotypic individuals, and so can act as barriers to gene flow between opposing karyotypes. The 2La inversion, which is approximately 21Mb long, has been associated with aridity tolerance, *Plasmodium* infection and insecticide resistance. Because the 2La inversion predates the speciation of the gambiae complex, it is the biggest driver of population structure within its breakpoints.

In *RNA-Seq-Pop*, we can estimate the frequency of chromosomal inversions in our samples, using [compkaryo](https://github.com/rrlove/compkaryo) and karyotype-tagging SNPs. These are SNPs which reside within the inversion breakpoints, and show fixed differences between karyotypes, indicating which karyotype a sample contains.

**Results**

In [45]:
import plotly.graph_objects as go

df = df_karyo.filter(like="freq")


fig = px.imshow(
    df,
    color_continuous_scale="OrRd",
    aspect="auto",
    text_auto=True,
    zmin=0,
    zmax=1,
    width=200 + (df.shape[1] * 150),
    title=f"{dataset} karyotype frequencies", 
)
fig.update(layout_coloraxis_showscale=False)
fig.write_image(f"{wkdir}/results/karyotype/karyotype_heatmap.png", scale=2)
fig.show()